In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, make_scorer, balanced_accuracy_score
from sklearn.pipeline import Pipeline


df_X_train = pd.read_csv('../data/raw/X_train.csv')
df_y_train = pd.read_csv('../data/raw/y_train.csv')
df_X_test = pd.read_csv('../data/raw/X_test.csv')

ImportError: cannot import name 'balanced_accuracy_score'

In [2]:
df_train = pd.merge( df_y_train, df_X_train, on='id')
df_train = pd.concat([df_train.loc[df_train['y'] == 1].iloc[:600], df_train.loc[df_train['y'] != 1]]).reset_index().drop('index', axis = 1)
print(df_train)

        id    y        x0        x1        x2        x3        x4        x5  \
0        0  1.0 -4.524480 -0.718917  0.827537 -2.336564  1.905992 -1.424985   
1        2  1.0 -0.547026 -0.045593  1.016072 -0.068002 -0.670472 -0.551299   
2        3  1.0 -1.939258 -0.284554  1.276007 -0.500731  1.088817 -0.897736   
3        4  1.0 -0.386835 -0.143997  0.506509 -0.648928 -0.614121  0.211504   
4        5  1.0 -1.933678 -0.645258  0.762559 -0.065472  0.930215 -0.576592   
5        7  1.0 -2.566073 -0.896698  0.390199 -0.448086  1.524759 -0.049982   
6        8  1.0 -4.037710 -0.995334  2.562033 -0.452974  1.742658 -2.414891   
7        9  1.0 -1.447890 -0.006329  0.103616 -1.350514 -0.217469 -0.584939   
8       11  1.0 -1.838458  0.324521  0.359177 -1.781189  3.102764 -0.202826   
9       13  1.0 -1.621783 -1.584052  1.529401 -1.005307  0.338492 -1.382978   
10      14  1.0 -2.999272 -0.847908  0.880497 -0.514934  1.020666 -0.905973   
11      16  1.0 -0.020872 -0.211064 -0.252272  0.933

In [3]:
X = df_train
Y = X['y'].values
X = X.drop('y', axis = 1).values

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state=42)

In [5]:
pipelines = []
pipelines.append(('ScaledADA', Pipeline([('Scaler', RobustScaler()),('ADA', AdaBoostClassifier())])))
pipelines.append(('ScaledGBM', Pipeline([('Scaler', RobustScaler()),('GBM', GradientBoostingClassifier())])))
pipelines.append(('ScaledGPR', Pipeline([('Scaler', RobustScaler()),('GPR', RandomForestClassifier())])))

In [11]:
results = []
names = []
scorer = make_scorer()
for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=42)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scorer)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

ValueError: Found input variables with inconsistent numbers of samples: [1800, 1440]

In [7]:
X_train = X
y_train = Y
X_test = df_X_test.values

In [8]:
scaler = RobustScaler().fit(X_train)
rescaled_X_train = scaler.transform(X_train)
rescaled_X_test = scaler.transform(X_test)

In [9]:
model = GradientBoostingClassifier()
model.fit(rescaled_X_train, y_train)
y_pred = model.predict(rescaled_X_test) #predicted values without index column
df_y_pred = pd.DataFrame({'id': np.array(df_X_test['id']),'y': y_pred})
df_y_pred.to_csv('../data/processed/y_pred.csv', index=False)

In [ ]:
param_grid = dict(n_classifiers=np.arange(1,200))
model = Pipeline([('GBM', GradientBoostingRegressor())])
kfold = KFold(n_splits=10, random_state=42)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='r2', cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))